# YOLO v1 on MAX78000

This notebook covers the MAX78000 model training and synthesis pipeline for the YOLO v1 model.

Prerequisites:
 - The `ai8x-training` folder must be located underneath a common root directory. Training and Synthesis repositories must be initialized and ready to use (please see [README.md](README.md)).
 - This notebook must be located inside the `ai8x-training` folder.
 - Place `yolo` folder into `ai8x-training`.
 

In [1]:
import importlib

import os
from matplotlib.image import imread
import numpy as np
import matplotlib.pyplot as plt
import time

import torch
from torch import nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import transforms
import distiller.apputils as apputils
import cv2

import sys
sys.path.insert(0, 'yolo/')
sys.path.insert(1, 'distiller/')
sys.path.insert(2, '/data/detection/')

from YOLO_V1_DataSet_small import YoloV1DataSet
from YOLO_V1_LossFunction import  Yolov1_Loss

mod = importlib.import_module("yolov1_bn_model_noaffine")

import ai8x
%matplotlib inline

# Part I - Training

## Utilities

In [2]:
def count_params(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    return params

In [3]:
class Args:
    def __init__(self, act_mode_8bit):
        self.act_mode_8bit = act_mode_8bit
        self.truncate_testset = False

## Model Training

In [4]:
# Data path, modify to match file system layout
data_path = '/data/detection'

In [5]:
dataSet = YoloV1DataSet(imgs_dir="/data/detection/dataset/train/VOC2007/Train/JPEGImages",
                        annotations_dir="/data/detection/dataset/train/VOC2007/Train/Annotations",
                        ClassesFile="/data/detection/dataset/train/VOC2007/Train/VOC_remain_class.data",
                        data_path=data_path)

#Unique images 486
486 486
{'person': 0, 'car': 1, 'bicycle': 2, 'chair': 3, 'sofa': 4}


In [6]:
dataLoader = DataLoader(dataSet,batch_size=16,shuffle=True,num_workers=4)

In [17]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

ai8x.set_device(device=85, simulate=False, round_avg=False)
Yolo = mod.Yolov1_net(num_classes=dataSet.Classes, bias=True)
Yolo = Yolo.to(device)
print("NUMBER OF PARAMETERS",  sum(p.numel() for p in Yolo.parameters()))

Running on device: cuda:0
Configuring device: MAX78000, simulate=False.
YOLO V1 Model_Z %d class (224 input), %d bounding boxes.
NUMBER OF PARAMETERS 296591


In [18]:
loss_function = Yolov1_Loss().to(device)

In [19]:
optimizer = optim.SGD(Yolo.parameters(),lr=3e-5,momentum=0.9,weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[50, 100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,20000,30000,40000],gamma=0.8)

In [20]:
num_epochs = 400
qat_policy = {'start_epoch':150,
              'weight_bits':8,
              'bias_bits':8,
              'shift_quantile': 0.99}

In [ ]:
# MAIN TRAINING

best_acc = 0
best_qat_acc = 0
for epoch in range(0, num_epochs):
    loss_sum = 0
    loss_coord = 0
    loss_confidence = 0
    loss_classes = 0
    epoch_iou = 0
    epoch_object_num = 0
    scheduler.step()
    
    if epoch > 0 and epoch == qat_policy['start_epoch']:
        print('QAT is starting!')
        # Fuse the BN parameters into conv layers before Quantization Aware Training (QAT)
        torch.save(Yolo.state_dict(), f'yolo_models/scaled224_noaffine_shift{qat_policy["shift_quantile"]}_maxim_yolo_beforeQAT_ep{epoch:04d}.pth')
        ai8x.fuse_bn_layers(Yolo)

        # Switch model from unquantized to quantized for QAT
        ai8x.initiate_qat(Yolo, qat_policy)

        # Model is re-transferred to GPU in case parameters were added
        Yolo.to(device)


    for batch_index, batch_train in enumerate(dataLoader):

        optimizer.zero_grad()
        train_data = batch_train[0].float().to(device)
        train_data.requires_grad = True
        
        label_data = batch_train[1].float().to(device)
        label_data[:, :, :, :, 5] = label_data[:, :, :, :, 5] / 224
        label_data[:, :, :, :, 6] = label_data[:, :, :, :, 6] / 224
        label_data[:, :, :, :, 7] = label_data[:, :, :, :, 7] / 224
        label_data[:, :, :, :, 8] = label_data[:, :, :, :, 8] / 224
        label_data[:, :, :, :, 9] = label_data[:, :, :, :, 9] / (224*224)
        
        
#         label_data = batch_train[1].float().to(device)
        bb_pred, _ = Yolo(train_data)
        loss = loss_function(bounding_boxes=bb_pred,ground_truth=label_data)
        batch_loss = loss[0]
        loss_coord = loss_coord + loss[1]
        loss_confidence = loss_confidence + loss[2]
        loss_classes = loss_classes + loss[3]
        epoch_iou = epoch_iou + loss[4]
        epoch_object_num = epoch_object_num + loss[5]
        batch_loss.backward()
        optimizer.step()
        batch_loss = batch_loss.item()
        loss_sum = loss_sum + batch_loss
        
        #print("batch_index : {} ; batch_loss : {}".format(batch_index, batch_loss))
    
    
    if epoch % 50 == 0:
        if epoch >= qat_policy['start_epoch']:
            torch.save(Yolo.state_dict(), f'yolo_models/scaled224_noaffine_shift{qat_policy["shift_quantile"]}_maxim_yolo_qat_ep{epoch:04d}.pth')
        else:
            torch.save(Yolo.state_dict(), f'yolo_models/scaled224_noaffine_shift{qat_policy["shift_quantile"]}_maxim_yolo_ep{epoch:04d}.pth')
            
        
    avg_loss= loss_sum/batch_index
    print("epoch : {} ; loss : {} ; avg_loss: {}".format(epoch,{loss_sum},{avg_loss}))
    
    epoch = epoch + 1
    
torch.save(Yolo.state_dict(), f'yolo_models/scaled224_noaffine_shift{qat_policy["shift_quantile"]}_maxim_yolo_qat_ep{epoch:04d}.pth')    

epoch : 0 ; loss : {11724.577575683594} ; avg_loss: {390.8192525227865}
epoch : 1 ; loss : {9242.66722869873} ; avg_loss: {308.088907623291}
epoch : 2 ; loss : {8296.884826660156} ; avg_loss: {276.5628275553385}
epoch : 3 ; loss : {7883.095397949219} ; avg_loss: {262.7698465983073}
epoch : 4 ; loss : {7646.734916687012} ; avg_loss: {254.89116388956705}
epoch : 5 ; loss : {7468.997764587402} ; avg_loss: {248.96659215291342}
epoch : 6 ; loss : {7296.804267883301} ; avg_loss: {243.22680892944337}
epoch : 7 ; loss : {7161.190208435059} ; avg_loss: {238.70634028116862}
epoch : 8 ; loss : {6985.4044189453125} ; avg_loss: {232.84681396484376}
epoch : 9 ; loss : {6830.99479675293} ; avg_loss: {227.699826558431}
epoch : 10 ; loss : {6681.527252197266} ; avg_loss: {222.7175750732422}
epoch : 11 ; loss : {6450.0129470825195} ; avg_loss: {215.00043156941732}
epoch : 12 ; loss : {6210.836292266846} ; avg_loss: {207.02787640889485}
epoch : 13 ; loss : {6101.955642700195} ; avg_loss: {203.39852142333

epoch : 113 ; loss : {549.1962213516235} ; avg_loss: {18.306540711720785}
epoch : 114 ; loss : {534.798632144928} ; avg_loss: {17.8266210714976}
epoch : 115 ; loss : {543.9386367797852} ; avg_loss: {18.131287892659504}
epoch : 116 ; loss : {524.4820432662964} ; avg_loss: {17.482734775543214}
epoch : 117 ; loss : {505.5712580680847} ; avg_loss: {16.852375268936157}
epoch : 118 ; loss : {507.30362701416016} ; avg_loss: {16.910120900472005}
epoch : 119 ; loss : {495.53395891189575} ; avg_loss: {16.517798630396523}
epoch : 120 ; loss : {493.3015456199646} ; avg_loss: {16.44338485399882}
epoch : 121 ; loss : {479.87072467803955} ; avg_loss: {15.995690822601318}
epoch : 122 ; loss : {475.9064989089966} ; avg_loss: {15.86354996363322}
epoch : 123 ; loss : {467.34638118743896} ; avg_loss: {15.578212706247966}
epoch : 124 ; loss : {475.29113149642944} ; avg_loss: {15.843037716547649}
epoch : 125 ; loss : {469.0866403579712} ; avg_loss: {15.636221345265707}
epoch : 126 ; loss : {446.928452491760

epoch : 224 ; loss : {509.20156383514404} ; avg_loss: {16.973385461171468}
epoch : 225 ; loss : {504.0981831550598} ; avg_loss: {16.803272771835328}
epoch : 226 ; loss : {500.4142212867737} ; avg_loss: {16.680474042892456}
epoch : 227 ; loss : {496.92449378967285} ; avg_loss: {16.564149792989095}
epoch : 228 ; loss : {494.4205515384674} ; avg_loss: {16.480685051282247}
epoch : 229 ; loss : {489.93157386779785} ; avg_loss: {16.33105246225993}
epoch : 230 ; loss : {485.75095415115356} ; avg_loss: {16.19169847170512}
epoch : 231 ; loss : {481.471875667572} ; avg_loss: {16.049062522252402}
epoch : 232 ; loss : {477.7222566604614} ; avg_loss: {15.92407522201538}
epoch : 233 ; loss : {474.3010640144348} ; avg_loss: {15.810035467147827}
epoch : 234 ; loss : {495.3179588317871} ; avg_loss: {16.510598627726235}
epoch : 235 ; loss : {503.94938468933105} ; avg_loss: {16.798312822977703}
epoch : 236 ; loss : {494.9067325592041} ; avg_loss: {16.496891085306803}
epoch : 237 ; loss : {485.62626409530